# deployment steps:
0) 
1) make sure that Dockerfile, requirements.txt and main.py is in repo
2) export vars to ~/.zshrc
!!!full paths to the credentials.json and token.json this is for local run
change here GOOGLE_CLOUD_PROJECT, GOOGLE_OAUTH_CLIENT_FILE, GOOGLE_OAUTH_TOKEN_FILE
```
export GOOGLE_GENAI_USE_VERTEXAI=True
export GOOGLE_CLOUD_PROJECT=personalplanner-476218       
export GOOGLE_CLOUD_LOCATION=us-central1
export GOOGLE_OAUTH_CLIENT_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/credentials.json"  
export GOOGLE_OAUTH_TOKEN_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/token.json"
export MODEL=gemini-2.5-flash
```
3) ```source ~/.zshrc```
NOTE to run the current verison of setup run with python main.py. the above exports are for local run only, not cloudrun!!!

4) deploy containers ( change project name in 2 places here)

```
gcloud run deploy project-planner-service \
  --source . \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --memory=1Gi \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json
```


5) run this ( Change project name )
```
gcloud run services describe project-planner-service \
  --region=us-central1 \
  --project=personalplanner-476218 \
  --format='value(spec.template.spec.serviceAccountName)'
```


it returns something like xxxxxxx-compute@developer.gserviceaccount.com
then run what's below with that thingy as your SA_EMAIL
```
SA_EMAIL="98380938461-compute@developer.gserviceaccount.com"      

gcloud projects add-iam-policy-binding personalplanner-476218 \
  --member="serviceAccount:${SA_EMAIL}" \
  --role="roles/aiplatform.user"

```
6) run this if session changed
```
gcloud run services update-traffic project-planner-service \
  --region us-central1 \
  --to-latest
```
7) idk anymore force cloud run to a single instance
```
gcloud run services update project-planner-service \
  --region us-central1 \
  --max-instances=1 \
  --min-instances=1

```

# Paste your service URL here 

In [ ]:
import requests
import json
# Service [project-planner-service] revision [project-planner-service-00002-fhq] has been deployed and is serving 100 percent of traffic.
# Service URL: https://project-planner-service-98380938461.us-central1.run.app
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"

# Helpers

In [29]:
APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# 2) List available apps
def list_apps():
    url = f"{SERVICE_URL}/list-apps"
    resp = requests.get(url)
    resp.raise_for_status()
    apps = resp.json()
    print("Available apps:")
    pretty_print(apps)
    return apps


# 3) Create or update a session

def create_session(initial_state=None):
    if initial_state is None:
        initial_state = {}

    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.post(url, json=initial_state)

    print("Status:", resp.status_code)
    print("Headers:", resp.headers)
    print("Body:\n", resp.text)

    # Only raise if it's 4xx/5xx *after* we print
    resp.raise_for_status()

    try:
        session = resp.json()
        print("\nParsed session JSON:")
        pretty_print(session)
        return session
    except Exception as e:
        print("\nJSON parse error:", e)
        return None



# 4) Get session (optional, for debugging)
def get_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.get(url)
    resp.raise_for_status()
    session = resp.json()
    print("Current session:")
    pretty_print(session)
    return session


# 5) Run the orchestrator with a user message
def run_orchestrator(message_text: str):
    url = f"{SERVICE_URL}/run"
    payload = {
        "app_name": APP_NAME,
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "new_message": {
            "role": "user",
            "parts": [
                {"text": message_text}
            ]
        }
    }
    resp = requests.post(url, json=payload)
    print("Status:", resp.status_code)
    print("Body:\n", resp.text)
    resp.raise_for_status()
    events = resp.json()
    print("Events from orchestrator:")
    pretty_print(events)
    return events


# 6) Convenience helper: full flow in one call
def send_to_orchestrator(message_text: str, init_if_needed: bool = True):
    """
    - Optionally creates/updates a session first
    - Sends a message
    - Returns events
    """
    if init_if_needed:
        # you can attach any initial session state here if you want
        create_session(initial_state={})

    return run_orchestrator(message_text)


# 7) Delete session (if you want to reset state)
def delete_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.delete(url)
    if resp.status_code not in (200, 204):
        print("Delete returned:", resp.status_code, resp.text)
    else:
        print("Session deleted.")



In [30]:
apps = list_apps()

Available apps:
[
  "apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "services",
  "ui",
  "utils"
]


In [31]:
create_session(initial_state={"note": "session created from Jupyter"})


Status: 200
Headers: {'content-type': 'application/json', 'x-cloud-trace-context': 'e659bdcedd0ca54ea8e752dd1ce3f1d4', 'date': 'Mon, 10 Nov 2025 07:42:04 GMT', 'server': 'Google Frontend', 'Content-Length': '146', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Body:
 {"id":"sess1","appName":"orchestrator","userId":"grant","state":{"note":"session created from Jupyter"},"events":[],"lastUpdateTime":1762760524.0}

Parsed session JSON:
{
  "id": "sess1",
  "appName": "orchestrator",
  "userId": "grant",
  "state": {
    "note": "session created from Jupyter"
  },
  "events": [],
  "lastUpdateTime": 1762760524.0
}


{'id': 'sess1',
 'appName': 'orchestrator',
 'userId': 'grant',
 'state': {'note': 'session created from Jupyter'},
 'events': [],
 'lastUpdateTime': 1762760524.0}

In [32]:
events = run_orchestrator(
    """what time is it right now?"""
)


Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"text":"The current time is 07:42:07 UTC on Monday, November 10, 2025.","thoughtSignature":"CqEDAePx_17B2AVJmbj75v5VmjNXkB-E6qWlHSpS9O6dxQtHAWQO7nHDKzChmwupzCDBNT3MTdJUWy5wwpbrJmy_vzNwbe0rLKnbpMaxMd9o4_hxntuRBLBTIl6BSeY8GVMufmOCTVPvxNTEPWiJq_miqKxnSaVayBLI6kBCaodqZ-SltCFiU1GD8umGtFuOR2XNqVvrjxgroSZ8T6t5uoLdyQbe2jx40iFCH9FP7-dYJVBear4cpXrkIqJibC4cLf3w4pKIfrvpQcO95pknnVQ0lyHPn_Khzf5FHWJavAe-_IQtscgXbvRyWxy0J9PbiV2ka7dJdFPa2XDwE9mTHUWLSRCKN5L36Kpw8Qe3GSRdT_jjIKAZcngaYLqzrn0NmqBeQWMPZBtdKlWX9pO4PKC4uajZyQeETGM30sLTzwpLlyJVdi2R_3-i9TGyryCkf7b4Lc7LCpgJI58UywwNWwmWXGh2mbj1p4EhGhc-dIz9ak4mqN31tVgVZFHEJmtXG1JzK2al7a7eEBWQjp_gQZ6E7AX_SYsR5-UW7GSJmJh4-pnj"}],"role":"model"},"finishReason":"STOP","usageMetadata":{"candidatesTokenCount":28,"candidatesTokensDetails":[{"modality":"TEXT","tokenCount":28}],"promptTokenCount":3392,"promptTokensDetails":[{"modality":"TEXT","tokenCount":3392}],"thoughtsTokenCount":104,"totalTokenC

In [3]:
import requests
import json
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"
create_session(initial_state={"note": "session created from Jupyter"})

APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))




run_orchestrator(
    """can you look for software engineer jobs posted for the past 7 days """
)

NameError: name 'create_session' is not defined

In [ ]:
run_orchestrator(
    """Tag jobs in the sheet that match 5 years experience in San Francisco, including remote jobs that qualify."""
)

# trash

In [10]:
from orchestrator_client import create_session, run_orchestrator, list_apps

list_apps()  # should show "orchestrator" in the list

create_session(initial_state={"note": "session created from REPL"})

run_orchestrator("can you look for software engineer jobs posted for the past 7 days")


Available apps:
[
  "apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "matching_service",
  "orchestrator",
  "personalplanner.egg-info",
  "resume_customization",
  "services",
  "ui",
  "utils"
]

[create_session]
Status: 409
Body: {"detail":"Session already exists: sess1"} 
Session already exists; reusing it.

[run_orchestrator]
Status: 500
Body: Internal Server Error 


HTTPError: 500 Server Error: Internal Server Error for url: https://project-planner-service-98380938461.us-central1.run.app/run

In [ ]:



create_session(initial_state={"note": "session from REPL"})
run_orchestrator("can you look for software engineer jobs posted for the past 7 days")
